In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define paths to your train and validation data
train_dir = '/Users/arshdeepsingh/Documents/pneumonia_detection/train'
val_dir = '/Users/arshdeepsingh/Documents/pneumonia_detection/val'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Change class_mode to 'categorical' for multiple classes
    shuffle=True
)

# Create a data generator for the validation set (no data augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Change class_mode to 'categorical' for multiple classes
    shuffle=False
)

# Calculate class weights for imbalanced data
def compute_class_weights(labels):
    class_weights = {}
    total_samples = len(labels)

    for class_label in np.unique(labels):
        class_count = np.sum(labels == class_label)
        class_weights[class_label] = total_samples / (len(np.unique(labels)) * class_count)

    return class_weights

class_weights = compute_class_weights(train_generator.classes)

# Convert integer labels to one-hot encoded labels
train_generator_labels_one_hot = to_categorical(train_generator.labels, num_classes=3)
val_generator_labels_one_hot = to_categorical(val_generator.labels, num_classes=3)

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Adjust the number of neurons for three classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    class_weight=class_weights
)


Found 5209 images belonging to 3 classes.
Found 23 images belonging to 3 classes.
Epoch 1/10
162/162 [==============================] - 51s 311ms/step - loss: 0.9352 - accuracy: 0.5177
Epoch 2/10
162/162 [==============================] - 51s 316ms/step - loss: 0.7083 - accuracy: 0.6936
Epoch 3/10
162/162 [==============================] - 52s 320ms/step - loss: 0.6625 - accuracy: 0.7006
Epoch 4/10
162/162 [==============================] - 53s 328ms/step - loss: 0.6227 - accuracy: 0.7302
Epoch 5/10
162/162 [==============================] - 54s 332ms/step - loss: 0.5862 - accuracy: 0.7358
Epoch 6/10
162/162 [==============================] - 54s 332ms/step - loss: 0.5852 - accuracy: 0.7388
Epoch 7/10
162/162 [==============================] - 54s 330ms/step - loss: 0.5608 - accuracy: 0.7452
Epoch 8/10
162/162 [==============================] - 53s 328ms/step - loss: 0.5618 - accuracy: 0.7512
Epoch 9/10
162/162 [==============================] - 54s 330ms/step - loss: 0.5602 - accuracy

In [7]:
model.save('pneumonia_detection_with_classes_model.h5')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
